# OpenClaw on Google Colab (Free CPU)

Run your own **OpenClaw** AI gateway entirely on a free Google Colab CPU instance.

**What this notebook does:**
1. Mounts your Google Drive for persistent data (SQLite databases, config, workspace)
2. Installs Node.js 22 and pnpm
3. Clones and builds OpenClaw from source
4. Lets you provide your LLM API key(s)
5. Starts the OpenClaw gateway with an ngrok tunnel so you can access it from anywhere

**Just run each cell top-to-bottom. That's it.**

> Your data persists in Google Drive at `MyDrive/openclaw/` — so reconnecting later picks up where you left off.

---
## Step 1: Mount Google Drive

This gives OpenClaw a persistent storage location for its SQLite databases, configuration, and workspace files. When the Colab runtime recycles, your data survives.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

# Persistent state directory on Google Drive
OPENCLAW_HOME = '/content/drive/MyDrive/openclaw'
OPENCLAW_STATE = os.path.join(OPENCLAW_HOME, 'state')   # ~/.openclaw equivalent
OPENCLAW_WORKSPACE = os.path.join(OPENCLAW_HOME, 'workspace')

os.makedirs(OPENCLAW_STATE, exist_ok=True)
os.makedirs(OPENCLAW_WORKSPACE, exist_ok=True)

print(f'State directory:     {OPENCLAW_STATE}')
print(f'Workspace directory: {OPENCLAW_WORKSPACE}')
print('Google Drive mounted and directories ready.')

---
## Step 2: Install Node.js 22 + pnpm

OpenClaw requires Node.js >= 22. Colab ships an older version, so we install a fresh one.

In [ ]:
%%bash
set -euo pipefail

# Check if Node 22+ is already installed (from a previous run in this session)
if command -v node &>/dev/null; then
  NODE_MAJOR=$(node -v | sed 's/v\([0-9]*\).*/\1/')
  if [ "$NODE_MAJOR" -ge 22 ]; then
    echo "Node.js $(node -v) already installed, skipping."
    echo "pnpm $(pnpm -v 2>/dev/null || echo 'not yet')"
    exit 0
  fi
fi

echo "Installing Node.js 22 via NodeSource..."
curl -fsSL https://deb.nodesource.com/setup_22.x | sudo -E bash - 2>&1 | tail -3
sudo apt-get install -y nodejs 2>&1 | tail -3

echo
echo "Enabling corepack + pnpm..."
sudo corepack enable
corepack prepare pnpm@latest --activate 2>/dev/null || true

echo
echo "Node.js $(node -v)"
echo "pnpm $(pnpm -v)"

---
## Step 3: Clone & Build OpenClaw

Clones the repo (if not already cloned), installs dependencies, and builds. This takes a few minutes on the first run. Subsequent runs reuse the existing clone and only rebuild if source changed.

In [ ]:
%%bash
set -euo pipefail

REPO_DIR="/content/openclaw"

if [ -d "$REPO_DIR/.git" ]; then
  echo "OpenClaw repo already cloned. Pulling latest..."
  cd "$REPO_DIR"
  git pull --ff-only 2>&1 | tail -5
else
  echo "Cloning OpenClaw..."
  git clone --depth 1 https://github.com/openclaw/openclaw.git "$REPO_DIR" 2>&1 | tail -5
  cd "$REPO_DIR"
fi

echo
echo "Installing dependencies (this may take a few minutes)..."
pnpm install --frozen-lockfile 2>&1 | tail -5

echo
echo "Building OpenClaw..."
pnpm build 2>&1 | tail -10

echo
echo "Building UI..."
pnpm ui:build 2>&1 | tail -5

echo
echo "Build complete."

---
## Step 4: Provide Your API Key(s)

Enter at least one LLM provider API key. Your key is stored in the Colab session (not saved to Drive unless you check the box below).

Supported providers: **Anthropic**, **OpenAI**, **Gemini**, **OpenRouter**

In [ ]:
import getpass
import os
import json

OPENCLAW_STATE = '/content/drive/MyDrive/openclaw/state'

print('Enter your LLM API key(s). Press Enter to skip a provider.\n')

keys = {}

anthropic_key = getpass.getpass('Anthropic API key (sk-ant-...): ')
if anthropic_key.strip():
    keys['ANTHROPIC_API_KEY'] = anthropic_key.strip()

openai_key = getpass.getpass('OpenAI API key (sk-...): ')
if openai_key.strip():
    keys['OPENAI_API_KEY'] = openai_key.strip()

gemini_key = getpass.getpass('Gemini API key: ')
if gemini_key.strip():
    keys['GEMINI_API_KEY'] = gemini_key.strip()

openrouter_key = getpass.getpass('OpenRouter API key (sk-or-...): ')
if openrouter_key.strip():
    keys['OPENROUTER_API_KEY'] = openrouter_key.strip()

if not keys:
    print('\n\u26a0\ufe0f  No API keys provided! OpenClaw needs at least one to function.')
    print('Re-run this cell to enter your key(s).')
else:
    # Set in current process environment
    for k, v in keys.items():
        os.environ[k] = v
    print(f'\n\u2705 {len(keys)} API key(s) configured: {", ".join(keys.keys())}')

# Generate a gateway token for this session
import secrets
gateway_token = secrets.token_hex(32)
os.environ['OPENCLAW_GATEWAY_TOKEN'] = gateway_token

# Write a .env file into the state dir so the gateway picks it up
env_path = os.path.join(OPENCLAW_STATE, '.env')
with open(env_path, 'w') as f:
    f.write(f'OPENCLAW_GATEWAY_TOKEN={gateway_token}\n')
    for k, v in keys.items():
        f.write(f'{k}={v}\n')

print(f'\nGateway token generated (for auth when connecting remotely).')
print(f'Env written to: {env_path}')

---
## Step 5: Start OpenClaw Gateway

This starts the gateway server. It binds to all interfaces (`--bind lan`) so the Colab proxy and ngrok can reach it.

The gateway runs as a background process so you can keep using the notebook.

**Dashboard URL** will be printed below. If you set up ngrok (optional step 6), you'll get a public URL too.

In [ ]:
import subprocess
import os
import time
import signal

REPO_DIR = '/content/openclaw'
OPENCLAW_STATE = '/content/drive/MyDrive/openclaw/state'
OPENCLAW_WORKSPACE = '/content/drive/MyDrive/openclaw/workspace'

# Build the environment
env = os.environ.copy()
env['OPENCLAW_STATE_DIR'] = OPENCLAW_STATE
env['OPENCLAW_WORKSPACE_DIR'] = OPENCLAW_WORKSPACE
env['HOME'] = '/content'
env['NODE_ENV'] = 'production'
# Skip channels that need external setup (WhatsApp, Signal, etc.)
env['OPENCLAW_SKIP_CHANNELS'] = '1'

# Symlink state dir to ~/.openclaw so the gateway finds it naturally
home_openclaw = os.path.expanduser('~/.openclaw')
if not os.path.exists(home_openclaw):
    os.symlink(OPENCLAW_STATE, home_openclaw)
    print(f'Symlinked {home_openclaw} -> {OPENCLAW_STATE}')

# Kill any previous gateway process
try:
    result = subprocess.run(['pkill', '-f', 'openclaw.mjs.*gateway'], capture_output=True)
    if result.returncode == 0:
        print('Stopped previous gateway process.')
        time.sleep(2)
except Exception:
    pass

# Start the gateway
log_file = open('/content/openclaw_gateway.log', 'w')
proc = subprocess.Popen(
    ['node', 'openclaw.mjs', 'gateway', '--allow-unconfigured', '--bind', 'lan', '--port', '18789'],
    cwd=REPO_DIR,
    env=env,
    stdout=log_file,
    stderr=subprocess.STDOUT,
    preexec_fn=os.setsid
)

print(f'Gateway starting (PID: {proc.pid})...')
time.sleep(5)

# Check if it's still running
if proc.poll() is None:
    print('\n\u2705 OpenClaw Gateway is running!')
    print(f'   Local URL: http://localhost:18789')
    print(f'   Logs: /content/openclaw_gateway.log')
    print(f'\n   Data stored in: Google Drive > MyDrive > openclaw/')
    print(f'   Gateway Token: {os.environ.get("OPENCLAW_GATEWAY_TOKEN", "N/A")[:12]}...')
else:
    print('\n\u274c Gateway failed to start. Check logs:')
    with open('/content/openclaw_gateway.log') as f:
        print(f.read()[-2000:])

---
## Step 6 (Optional): Expose via ngrok

If you want to access OpenClaw from outside Colab (e.g., connect messaging channels, use from your phone), set up an ngrok tunnel.

1. Get a free ngrok auth token at https://ngrok.com
2. Paste it below

In [ ]:
import getpass

ngrok_token = getpass.getpass('ngrok auth token (or press Enter to skip): ')

if ngrok_token.strip():
    # Install pyngrok
    import subprocess
    subprocess.run(['pip', 'install', 'pyngrok', '-q'], check=True)

    from pyngrok import ngrok, conf

    # Set auth token
    conf.get_default().auth_token = ngrok_token.strip()

    # Open tunnel
    tunnel = ngrok.connect(18789, 'http')
    print(f'\n\u2705 ngrok tunnel active!')
    print(f'   Public URL: {tunnel.public_url}')
    print(f'\n   Use this URL to connect to your OpenClaw gateway from anywhere.')
    print(f'   You\'ll need your Gateway Token to authenticate.')
else:
    print('Skipped ngrok. Gateway is only accessible within this Colab session.')
    print('You can use the Gradio UI in the next step to interact locally.')

---
## Step 7 (Optional): Simple Chat UI (Gradio)

A lightweight chat interface right here in the notebook. No external access needed.

In [ ]:
!pip install gradio websocket-client requests -q

import gradio as gr
import requests
import os
import json

GATEWAY_URL = 'http://localhost:18789'
TOKEN = os.environ.get('OPENCLAW_GATEWAY_TOKEN', '')


def check_gateway_health():
    """Check if the gateway is responding."""
    try:
        r = requests.get(f'{GATEWAY_URL}/health', timeout=5)
        return r.status_code == 200
    except Exception:
        return False


def send_message(message, history):
    """Send a message to the OpenClaw gateway via its HTTP API."""
    if not check_gateway_health():
        return 'Gateway is not running. Please run Step 5 first.'

    try:
        headers = {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {TOKEN}',
        }
        payload = {
            'message': message,
            'channel': 'webchat',
        }
        r = requests.post(
            f'{GATEWAY_URL}/api/chat',
            headers=headers,
            json=payload,
            timeout=120,
        )
        if r.status_code == 200:
            data = r.json()
            return data.get('reply', data.get('message', str(data)))
        else:
            return f'Error {r.status_code}: {r.text[:500]}'
    except requests.exceptions.Timeout:
        return 'Request timed out. The model may be processing a complex query.'
    except Exception as e:
        return f'Error: {str(e)}'


# Build the Gradio UI
with gr.Blocks(title='OpenClaw Chat') as demo:
    gr.Markdown('## OpenClaw Chat')
    gr.Markdown('Chat with your OpenClaw gateway directly from the notebook.')

    chatbot = gr.ChatInterface(
        fn=send_message,
        type='messages',
        examples=[
            'Hello! What can you do?',
            'What model are you using?',
            'Summarize the latest news about AI.',
        ],
    )

demo.launch(share=False, debug=False)

---
## Utilities

Helper cells for monitoring and managing the running gateway.

In [ ]:
# View recent gateway logs
!tail -50 /content/openclaw_gateway.log

In [ ]:
# Check if gateway is still running
!pgrep -fa 'openclaw.mjs.*gateway' || echo 'Gateway is NOT running. Re-run Step 5.'

In [ ]:
# Stop the gateway
!pkill -f 'openclaw.mjs.*gateway' && echo 'Gateway stopped.' || echo 'No gateway process found.'

In [ ]:
# Check what's stored on Google Drive
import os
openclaw_drive = '/content/drive/MyDrive/openclaw'
for root, dirs, files in os.walk(openclaw_drive):
    level = root.replace(openclaw_drive, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    if level < 3:  # Don't go too deep
        subindent = ' ' * 2 * (level + 1)
        for file in files[:20]:  # Cap at 20 files per dir
            size = os.path.getsize(os.path.join(root, file))
            print(f'{subindent}{file} ({size:,} bytes)')